In [9]:
import pandas as pd
import psycopg2



In [10]:
# Import csv file and read it into a dataframe
csv_file_path = '../final_files_proj3/Resources/ev_data.csv'
df =pd.read_csv(csv_file_path)

# Inspect the first or last few rows of the dataset to see that the data loaded properly and what you're working with
df.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00039
1,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
2,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.00000


In [11]:
# get a better idea of the dataset
df.describe()

,year,value
count,12654.000000,1.265400e+04
mean,2019.822112,4.273742e+05
std,5.476494,6.860498e+06
min,2010.000000,1.200000e-06
25%,2016.000000,2.000000e+00
50%,2020.000000,1.300000e+02
75%,2022.000000,5.500000e+03
max,2035.000000,4.400000e+08


In [12]:
# Check how many non null values each column has
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12654 entries, 0 to 12653
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   region      12654 non-null  object 
 1   category    12654 non-null  object 
 2   parameter   12654 non-null  object 
 3   mode        12654 non-null  object 
 4   powertrain  12654 non-null  object 
 5   year        12654 non-null  int64  
 6   unit        12654 non-null  object 
 7   value       12654 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 791.0+ KB


In [13]:
# how many rows have data
df.count()

region        12654
category      12654
parameter     12654
mode          12654
powertrain    12654
year          12654
unit          12654
value         12654
dtype: int64

In [14]:
# find the datatypes
df.dtypes

region         object
category       object
parameter      object
mode           object
powertrain     object
year            int64
unit           object
value         float64
dtype: object

In [15]:
# Check for missing values in the dataset
print(df.isnull().sum())

region        0
category      0
parameter     0
mode          0
powertrain    0
year          0
unit          0
value         0
dtype: int64


In [16]:
# Remove duplicate rows
df = df.drop_duplicates()

In [17]:
#Check dataset to see if any duplicates were dropped
df.describe()

,year,value
count,12654.000000,1.265400e+04
mean,2019.822112,4.273742e+05
std,5.476494,6.860498e+06
min,2010.000000,1.200000e-06
25%,2016.000000,2.000000e+00
50%,2020.000000,1.300000e+02
75%,2022.000000,5.500000e+03
max,2035.000000,4.400000e+08


In [18]:
# list the  countries/regions where we have data
df['region'].unique()

array(['Australia', 'Austria', 'Belgium', 'Brazil', 'Bulgaria', 'Canada',
       'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Estonia', 'EU27', 'Europe',
       'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland',
       'India', 'Indonesia', 'Ireland', 'Israel', 'Italy', 'Japan',
       'Korea', 'Latvia', 'Lithuania', 'Luxembourg', 'Mexico',
       'Netherlands', 'New Zealand', 'Norway', 'Poland', 'Portugal',
       'Rest of the world', 'Romania', 'Seychelles', 'Slovakia',
       'Slovenia', 'South Africa', 'Spain', 'Sweden', 'Switzerland',
       'Thailand', 'Turkiye', 'United Arab Emirates', 'United Kingdom',
       'USA', 'World'], dtype=object)

In [19]:
# list the unique values in the parameter column
df['parameter'].unique()

array(['EV stock share', 'EV sales share', 'EV sales', 'EV stock',
       'EV charging points', 'Electricity demand', 'Oil displacement Mbd',
       'Oil displacement, million lge'], dtype=object)

In [20]:
# list the unique values in the category column
df['category'].unique()

array(['Historical', 'Projection-STEPS', 'Projection-APS'], dtype=object)

In [21]:
# list the unique values in the mode column
df['mode'].unique()

array(['Cars', 'EV', 'Buses', 'Vans', 'Trucks'], dtype=object)

In [22]:
# list the unique values in the powertrain column
df['powertrain'].unique()

array(['EV', 'BEV', 'PHEV', 'Publicly available fast',
       'Publicly available slow', 'FCEV'], dtype=object)

In [23]:
# list the unique values in the unit column
df['unit'].unique()

array(['percent', 'Vehicles', 'charging points', 'GWh',
       'Milion barrels per day', 'Oil displacement, million lge'],
      dtype=object)

In [24]:
# Make dataframe of ev adoption from 2011 - 2023 by ev sales share 
df_evsales_share = df.loc[
    (df['year'] >= 2011) &
    (df['year'] <= 2023) &
    (df['category']== 'Historical') &
    (df['parameter']== 'EV sales share') 
]
df_evsales_share.head()

,region,category,parameter,mode,powertrain,year,unit,value
1,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.0065
6,Australia,Historical,EV sales share,Cars,EV,2012,percent,0.0300
13,Australia,Historical,EV sales share,Cars,EV,2013,percent,0.0340
18,Australia,Historical,EV sales share,Cars,EV,2014,percent,0.1600
25,Australia,Historical,EV sales share,Cars,EV,2015,percent,0.2000


In [25]:
# Filter data for fast and slow charging points (excluding other powertrains)
df_charging = df.loc[
    (df['powertrain'].isin(['Publicly available fast', 'Publicly available slow'])) &
    (df['parameter'] == 'EV charging points') &
    (df['category'] == 'Historical') &
    (df['unit'] == 'charging points') &
    (df['year'] >= 2011) & (df['year'] <= 2023)
]

# Group by region, year, and powertrain (fast vs. slow) and sum the values
df_combined_charging = df_charging.groupby(['region', 'year', 'powertrain'], as_index=False)['value'].sum()

# Step 3: Pivot the data to create separate columns for fast and slow charging points
df_pivot_charging = df_combined_charging.pivot_table(
    index=['region', 'year'],
    columns='powertrain',
    values='value',
    aggfunc='sum'
).reset_index()

# Fill any missing values (if there are any regions with no data for a specific powertrain)
df_pivot_charging = df_pivot_charging.fillna(0)

# Create the Total column by summing the fast and slow charging points
df_pivot_charging['Total'] = df_pivot_charging['Publicly available fast'] + df_pivot_charging['Publicly available slow']

# Remove the multi-level column index (caused by the pivot)
df_pivot_charging.columns.name = None  # This removes the "powertrain" label

# Reset the index to make it a clean dataframe without the index column
df_pivot_charging = df_pivot_charging.reset_index(drop=True)

# Sort the data by region and year
df_pivot_charging = df_pivot_charging.sort_values(by=['region', 'year'])

# Display the final dataframe without an index
print(df_pivot_charging.to_string(index=False))


        region  year  Publicly available fast  Publicly available slow     Total
     Australia  2017                     40.0                    440.0     480.0
     Australia  2018                     61.0                    670.0     731.0
     Australia  2019                      0.0                   1700.0    1700.0
     Australia  2020                      0.0                   2300.0    2300.0
     Australia  2021                    320.0                   2000.0    2320.0
     Australia  2022                    470.0                   2100.0    2570.0
     Australia  2023                    660.0                   2100.0    2760.0
       Austria  2011                      0.1                      0.1       0.2
       Austria  2012                      0.1                      0.1       0.2
       Austria  2013                      0.1                      0.1       0.2
       Austria  2014                      0.1                      0.1       0.2
       Austria  2015        

In [26]:
# Combine data for all vehicles (cars, trucks, vans, buses)
df_combined_vehicles = df.loc[
    (df['year'] >= 2011) & 
    (df['year'] <= 2023)
]

# Group by region, year, and parameter, summing the values for each mode (cars, trucks, vans, buses)
df_combined_vehicles_grouped = df_combined_vehicles.groupby(
    ['region', 'year', 'parameter'], as_index=False)['value'].sum()

# Optionally, filter for the parameter you want (e.g., EV sales share or EV stock share)
df_combined_vehicles_filtered = df_combined_vehicles_grouped[
    df_combined_vehicles_grouped['parameter'] == 'EV sales share'
]

# Display the result
df_combined_vehicles_filtered


,region,year,parameter,value
1,Australia,2011,EV sales share,0.006500
5,Australia,2012,EV sales share,0.030000
9,Australia,2013,EV sales share,0.034000
13,Australia,2014,EV sales share,0.160000
17,Australia,2015,EV sales share,0.200000
...,...,...,...,...
2635,World,2019,EV sales share,7.970000
2643,World,2020,EV sales share,30.029999
2651,World,2021,EV sales share,43.259999
2659,World,2022,EV sales share,65.220000


In [28]:
# Merge the charging data with the EV sales share data on region (and optionally on year)
merged_data = pd.merge(df_pivot_charging, df_combined_vehicles_filtered, 
                       on='region', how='inner')

# Perform a regression analysis, e.g., using statsmodels or scikit-learn
import statsmodels.api as sm

# Set x and y axis data
X = merged_data['total_charging_points']
y = merged_data['value']

# Add a constant for the intercept in the regression
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Print the regression results
print(model.summary())

KeyError: 'total_charging_points'

In [ ]:
df_2023_evsales_ordered = df_2023_evsales.sort_values(by='value', ascending = False)
df_2023_evsales_ordered.head(25)

more granual approach to analysis of the correlation between EV adoption and charging points availability

In [136]:
# Filter for EV charging points and EV sales data
charging_data = df[df['parameter'] == 'EV charging points']
vehicle_data = df[df['parameter'] == 'EV sales share']

# Break down charging data into fast and slow charging types
fast_charging = charging_data[charging_data['mode'] == 'Publicly available fast']
slow_charging = charging_data[charging_data['mode'] == 'Publicly available slow']

# Break down vehicle sales data by mode (vehicle types like cars, trucks, vans, buses)
vehicle_sales_cars = vehicle_data[vehicle_data['mode'] == 'Car']
vehicle_sales_trucks = vehicle_data[vehicle_data['mode'] == 'Truck']
vehicle_sales_vans = vehicle_data[vehicle_data['mode'] == 'Van']
vehicle_sales_buses = vehicle_data[vehicle_data['mode'] == 'Bus']

# Optionally: You can group and sum the values by 'region' and 'year' for each category

# Aggregate charging points data by region and year (for fast and slow)
fast_charging_summary = fast_charging.groupby(['region', 'year']).agg({'value': 'sum'}).reset_index()
slow_charging_summary = slow_charging.groupby(['region', 'year']).agg({'value': 'sum'}).reset_index()

# Aggregate vehicle sales data by region and year (for each vehicle type)
vehicle_sales_cars_summary = vehicle_sales_cars.groupby(['region', 'year']).agg({'value': 'sum'}).reset_index()
vehicle_sales_trucks_summary = vehicle_sales_trucks.groupby(['region', 'year']).agg({'value': 'sum'}).reset_index()
vehicle_sales_vans_summary = vehicle_sales_vans.groupby(['region', 'year']).agg({'value': 'sum'}).reset_index()
vehicle_sales_buses_summary = vehicle_sales_buses.groupby(['region', 'year']).agg({'value': 'sum'}).reset_index()

# Merge all the data back together to have a comprehensive dataset
charging_summary = pd.merge(fast_charging_summary, slow_charging_summary, on=['region', 'year'], how='outer', suffixes=('_fast', '_slow'))
vehicle_sales_summary = pd.merge(vehicle_sales_cars_summary, vehicle_sales_trucks_summary, on=['region', 'year'], how='outer', suffixes=('_car', '_truck'))
vehicle_sales_summary = pd.merge(vehicle_sales_summary, vehicle_sales_vans_summary, on=['region', 'year'], how='outer', suffixes=('', '_van'))
vehicle_sales_summary = pd.merge(vehicle_sales_summary, vehicle_sales_buses_summary, on=['region', 'year'], how='outer', suffixes=('', '_bus'))

# Merge the charging and vehicle sales data together
full_data = pd.merge(charging_summary, vehicle_sales_summary, on=['region', 'year'], how='outer')

# Check the merged dataset
full_data.head()

,value_fast,value_slow,value_car,value_truck,value,region,year,value_bus


In [137]:
import statsmodels.api as sm

# Prepare the independent (X) and dependent (y) variables
X = full_data[['value_fast', 'value_slow', 'value_car', 'value_truck', 'value_van', 'value_bus']]  # Independent variables
y = full_data['value']  # Dependent variable (EV sales share)

# Add a constant (intercept) to the independent variables
X = sm.add_constant(X)

# Perform OLS regression
model = sm.OLS(y, X).fit()

# Print the regression results
print(model.summary())


KeyError: "['value_van'] not in index"

More debugging 

In [ ]:
# Filter out rows where mode is 'EV'
df_filtered = df[df['mode'] != 'EV']

# Verify the unique modes
df_filtered['mode'].unique()


In [ ]:
# Aggregate the data for each mode and create separate value columns
df_aggregated = df_filtered.groupby(['region', 'year', 'mode'], as_index=False)['value'].sum()

# Pivot the table to separate each vehicle type into its own column
df_pivot = df_aggregated.pivot_table(index=['region', 'year'], columns='mode', values='value', aggfunc='sum', fill_value=0)

# Reset column names to match the required format
df_pivot = df_pivot.reset_index()

# Verify the new structure of the dataframe
df_pivot.head(55)


In [ ]:
# Assuming you have already aggregated the charging points into 'value_fast' and 'value_slow'
df_final = pd.merge(df_pivot, df_pivot_charging[['region', 'year', 'value_fast', 'value_slow']], on=['region', 'year'], how='left')

# Verify the final dataset
df_final.head()
